# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Enter the following parameters to install packages to the correct conda instance and environment.


`jupyter_conda_path : str`
    - The path to miniconda that this Jupyter notebook is running from. Do not include a terminal `/` at the end of the path.
    
`jupyter_conda_env : str`
    - The conda environment that this Jupyter notebook is running in

In [1]:
jupyter_conda_path = "/home/jgreen/.miniconda3c"
jupyter_conda_env = "jupyter"

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in the specified environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

**NOTE: If you recieve an import error for `jupyter-ui-poll`, you will have to manually install the package in a user container terminal with the following commands:**
```
source <jupyter_conda_path>/etc/profile.d/conda.sh
conda activate <jupyter_conda_env>
pip install jupyter-ui-poll
```

In [2]:
import os
import json
import sys

print('Checking conda environment for UI depedencies...')
os.system("bash " + os.getcwd() + f"/jupyter-helpers/install_ui_packages.sh {jupyter_conda_path} {jupyter_conda_env}")
print('All dependencies installed.')

sys.path.insert(0, os.getcwd() + '/jupyter-helpers')
import ui_helpers as ui
import pandas as pd

Checking conda environment for UI depedencies...
All dependencies installed.


## Step 1: Define Workflow Inputs

Run the following cells to generate interactive widgets allowing you to enter all workflow inputs. **All inputs must be filled out to proceed with the benchmarking process.**

### Cloud Resources

#### Compute Resources

Before defining anything else, the resources you intend to use with the benchmarking must be defined. Currently, only resources defined in the Parallel Works platform may be used. Also of note are options that will be passed to Dask: you must specify the number of cores and memory per worker node in the cluster. Without these values, Dask will not be able to submit jobs.

In particular, these options are included so that you can form fair comparisons between different cloud service providers (CSPs). Generally, different CSPs won't have worker nodes with the exact same specs, and in order to achieve a fair comparison between two CSPs one cluster will have to limited to not exceed to the computational power of the another.

<div class="alert alert-block alert-info">
For resources controlled from the Parallel Works platform, the <code>Resource name</code> box should be populated with the name found on the <b>RESOURCES</b>  tab.
    </div>

In [3]:
resource = ui.resourceWidgets()
resource.display()
resources = resource.processInput()
print(f'Your resource inputs:\n {resources}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Resource Name: '), Text(value=''))), HBox(child…

Button(description='Submit', style=ButtonStyle())


-----------------------------------------------------------------------------
If you wish to change information about cloud resources, run this cell again.

Your resource inputs:
 [{'Name': 'gcptestnew', 'CSP': 'GCP', 'Dask': {'Scheduler': 'SLURM', 'Partition': 'compute', 'CPUs': 2, 'Memory': 16.0}}]


#### Object Stores

This set of inputs is where you enter the cloud object store Universal Resource Identifiers (URIs). Both public and private buckets are supported. For the latter, ensure that you have access credentials with *at least* read, write, list, and put (copy from local storage to cloud) permissions, as format conversions will need to be made during the benchmarking process.

Be sure to double-check your inputs to ensure that the bucket names and credentials are correct!

In [4]:
store = ui.storageWidgets()
store.display()
storage = store.processInput()

# Following line should be commented out if you don't want AWS credentials shown in plain text
print(f'Your storage inputs:\n {storage}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Storage URI: '), Text(value='', placeholder='gc…

Button(description='Submit', style=ButtonStyle())


--------------------------------------------------------------------------------------
If you wish to change information about cloud storage locations, run this cell again.

Your storage inputs:
 [{'Path': 'gs://cloud-data-benchmarks', 'Type': 'Private', 'CSP': 'GCP', 'Credentials': {'token': './.cloud-data-benchmarks.json'}}]


### Datasets

#### User-Supplied Datasets

Below you can specify datasets that you want to be tested in the benchmarking. You can either enter single files or multiple files that belong to a single dataset, but that dataset match at least one of the supported formats. **Read the following input rules after running the UI cell below this one.**


1. Activate the checkbox if you desire to record your user-defined datasets. If it is not checked, none of your inputs will be recorded.

<br>

2. If speciying data in a bucket, input the path to the data within the bucket, *not* the full URI (i.e., use `path/to/file.extension` and not `<URI prefix>://<bucketname>/path/to/file.extension`)

<br>

3. Use absolute paths for data stored in the user container (or a filesystem mounted in the user container): `/path/to/data.extension`

<br>

4. Use globstrings (`path/to/files/*`) to specify datasets that are split up into multiple subfiles.
    - If using a globstring, ensure that *only* files that belong to the dataset exist in that directory. The workflow will take all files in the directory before the `*` and attempt to gather them into a single dataset.
    - **Globstrings are NOT supported for NetCDF files**
    
<br>

5. If you have a dataset stored in multiple cloud storage locations that will be used in the benchmarking, you must input the full URI of that dataset for each of these locations. That is, you must define each location of the data separately.

In [5]:
userdata = ui.userdataWidgets(storage=storage)
userdata.display()
user_files = userdata.processInput()
print(f'Your data inputs:\n {user_files}')

Checkbox(value=False, description='Provide datasets to workflow?')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Data Format'), Dropdown(options=('NetCDF4', 'CS…

Button(description='Submit', style=ButtonStyle())


---------------------------------------------------------------------------
If you wish to change information about your input data, run this cell again.

Your data inputs:
 [{'Format': 'NetCDF4', 'SourcePath': 'gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.nc', 'DataVars': ['*'], 'Type': 'Private', 'CSP': 'GCP', 'Credentials': {'token': './.cloud-data-benchmarks.json'}}]


#### Randomly-Generated Datasets

Another option to supply data to the benchmarking is to create randomly-generated datasets. These sets can be as large as you want (as they are written in parallel), and provide a great option if you are new to the world of cloud-native data formats. There are currently two supported randomly-generated data formats: CSV and NetCDF4. Since NetCDF4 is a gridded data format, an option to specify the number of coordinate axes is also included.

<div class="alert alert-block alert-info">
Randomly-generated NetCDF4 file sizes are limited by available disk space in the cluster you are generating the file with. Ensure that you have adequate disk space in your cluster, or the file will not fully generate.
    </div>

In [6]:
randgen = ui.randgenWidgets(resources=resources)
randgen.display()
randfiles = randgen.processInput()
print(f'Your randomly-generated file options:\n {randfiles}')

Accordion(children=(HBox(children=(Label(value='Resource to write random files with: '), Dropdown(options=('gc…

Button(description='Submit', style=ButtonStyle())


-------------------------------------------------------------------------------
If you wish to change the randomly generated file options, run this cell again.

Your randomly-generated file options:
 [{'Format': 'CSV', 'Generate': False, 'SizeGB': 0.0}, {'Format': 'NetCDF4', 'Generate': True, 'SizeGB': 3.0, 'Data Variables': 1.0, 'Float Coords': 2.0}, {'Resource': 'gcptestnew'}]


### Benchmark Options

#### Legacy to Cloud-Native Conversion Options

There are a number of different options to choose from when legacy files (e.g., CSV, NetCDF) are converted to cloud-native (e.g., Parquet, Zarr). In fact, this input field is where you have the most control over the type of resutls you want to see. You may specify different compression algorithms, compression levels, and chunksizes to use when cloud-native files are written to cloud storage, which each have different effects on how fast files will be written and read.

Note that, for large chunksizes, the workflow may fail. This is an intended consequence, as the workflow is designed to let you explore the effects of different options on your data. **In general, it is recommended that you keep chunksizes in the range of 50-150 megabytes**. Anything higher will result in either poor performance or out-of-memory errors. Similarly, some compression algorithms will result in poor performance.

The widget below allows you to define different sets of options to convert files with. To make multiple selections in the compression algorithm and dataset fields, simply hold `SHIFT` and/or `CTRL` (or `COMMAND` on Mac). Using these option sets, the files defined in each set will be converted using the specified compression & chunksize information. The following rules apply:

- An individual dataset will be converted `(# of compression algorithms in option set 1)*...*(# of compression algorithms in option set n)` times
- The total number of conversions is equal to the sum of individual dataset conversions
- Datasets will only be converted if they are selected in at least one option set.
- **If you want to keep the internal chunking scheme of the NetCDF data, set chunksize to 0**

Gridded data chunk dimensions are computed as n-dimensional cubes (i.e., a data variable described by 3 dimensions will have a 3-dimensional chunk of uniform dimension lengths, and so on for higher-order cases) 

In [7]:
convertOptions = ui.convertOptions(user_files, randfiles)
convertOptions.display()
convert_options = convertOptions.processInput()
print(f'Your cloud-native data options:\n {convert_options}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Data Format Type: '), Dropdown(options=('Gridde…

Button(description='Submit', style=ButtonStyle())


---------------------------------------------------------------------------
If you wish to change information about your conversion options, run this cell again.

Your cloud-native data options:
 [{'Algorithms': ('lz4', 'lz4hc'), 'Level': 5, 'Chunksize': 0.0, 'Datasets': ('random_3.0GB_NetCDF4.nc', 'ETOPO1_Ice_g_gmt4.nc')}, {'Algorithms': ('lz4',), 'Level': 5, 'Chunksize': 150.0, 'Datasets': ('ETOPO1_Ice_g_gmt4.nc',)}]


#### Core Options

Finally, there are a couple important options that apply to the core of the benchmarks. These are:

1. `worker_step : int` - When performing file reads (option not yet implemented in conversions), the workflow will loop through a range of Dask workers (or number of parallel reads). The loop starts at the maximum defined worker amount, and reduces by `worker_step` until the lowest possible value of workers is reached.

1. `tests : int` - The number of times each individual file will be read for measurement. Entering a number greater than 1 will take much longer to run, but results will reflect the volaility in network performance you can expect when using the data for computations.

In [8]:
worker_step = 5
tests = 1

## Step 2: Notebook Setup

Executing the following cell will write all of your inputs to `inputs.json`, install miniconda3 and the "cloud-data" Python environment to all resources, and write randomly-generated files to all cloud storage locations (if any files were specified). If writing randomly-generated files, especially large ones, the execution of this cell may take a while.

<div class="alert alert-block alert-info">
While randomly-generated files are written in parallel by default, if you wish to speed up the execution of this cell, consider creating/choosing a resource with more powerful worker nodes.
    </div>

In [44]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
                         "STORAGE" : storage,
                         "USERFILES" : user_files,
                         "RANDFILES" : randfiles,
                         "CONVERTOPTS" : convert_options,
                         "GLOBALOPTS" : {'worker_step' : worker_step, 'tests' : tests}
                        })

with open('inputs.json', 'w') as outfile:
    outfile.write(user_input)

os.system("bash workflow_notebook_setup.sh")

print('Workflow setup complete.')

Setting up workflow...
Will install miniconda3 to "/home/jgreen/.miniconda3"
Installing Miniconda-latest on "gcptestnew"...
Miniconda is already installed in "/home/jgreen/.miniconda3"!
Finished installing Miniconda on "gcptestnew".

 
Building "cloud-data" environment on "gcptestnew"...
Environment already exists!
Finished building "cloud-data" environment on "gcptestnew".

Done installing Miniconda-latest and building `cloud-data` on all requested resources.


Generating random files (this will take a while for large size requests)...
Generating NetCDF4...
Writing data variable "random1"...
Data variable "random1" written to "./random_3.0GB_NetCDF4.nc"
Uploading "./random_3.0GB_NetCDF4.nc" to "gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/randfiles/random_3.0GB_NetCDF4.nc"...
File written to "gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/randfiles/random_3.0GB_NetCDF4.nc"
Done.


2023-08-10 22:24:16,812 - distributed.deploy.adaptive_core - INFO - Adaptive stop
2023-08-10 22:24:16,954 - distributed.deploy.adaptive_core - INFO - Adaptive stop


Workflow setup complete.


## Step 3: Run Benchmarking

### Convert File to Cloud-Native

Since one of the major goals of this benchmarking is testing legacy formats against cloud-native ones, we must convert your legacy-formatted data (CSV and NetCDF4) into their corresponding cloud-native formats. This cell will execute and time the conversion process, writing each new format in parallel. The conversion will be done using each cluster's full amount of resources, so be mindful of this feature when using clusters that are expensive to operate.

In [50]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"convert-data.py\" \"conversions.csv\"")
df_conversions = pd.read_csv(os.getcwd() + '/results/csv-files/conversions.csv')
df_conversions

Waiting for worker nodes to start up...
Workers active.


Converting files in "gs://cloud-data-benchmarks" with "gcptestnew"...

Converting data variables Z1 from ETOPO1_Ice_g_gmt4.nc with 0.0MB chunks & level 5 lz4 compression to Zarr...
Written to "gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr"
Converting data variables Z1 from ETOPO1_Ice_g_gmt4.nc with 0.0MB chunks & level 5 lz4hc compression to Zarr...
Written to "gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4hc_5.zarr"
Converting data variables Z1 from ETOPO1_Ice_g_gmt4.nc with 150.0MB chunks & level 5 lz4 compression to Zarr...
Written to "gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_150.0MB_lz4_5.zarr"
Converting data variables random1 from random_3.0GB_NetCDF4.nc with 0.0MB chunks & level 5 lz4 compression to Zarr...
Written to "gs://cloud-data-ben

,resource,resource_csp,bucket,bucket_csp,conversionType,original_dataset_name,data_variables,conversion_time
0,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,NetCDF-to-Zarr,ETOPO1_Ice_g_gmt4.nc,*,9.302547
1,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,NetCDF-to-Zarr,ETOPO1_Ice_g_gmt4.nc,*,6.441293
2,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,NetCDF-to-Zarr,ETOPO1_Ice_g_gmt4.nc,*,5.708016
3,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,NetCDF-to-Zarr,random_3.0GB_NetCDF4.nc,*,18.860593
4,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,NetCDF-to-Zarr,random_3.0GB_NetCDF4.nc,*,16.569385


### File Reads

The last computation-intensive test in the benchmarking is reading and timing files from cloud storage. This will give you an idea of what data transfer throughput you can expect when using cloud storage and different data formats in other workflows.

In [ ]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"read-data.py\" \"reads.csv\"")
df_reads = pd.read_csv(os.getcwd() + '/results/csv-files/reads.csv')
df_reads

## TODO: Step 4: Visualize Results

**Feature not ready**

## Step 5: Remove Benchmarking Files from Cloud Resources (Optional)

In [53]:
os.system("bash postprocessing/remove-benchmark-files.sh")

Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/.zattrs#1691706641280223...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/.zgroup#1691706640643778...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/.zmetadata#1691706642598058...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/Z1/.zattrs#1691706642247161...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/Z1/.zarray#1691706641982713...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/ETOPO1_Ice_g_gmt4.nc_0.0MB_lz4_5.zarr/Z1/0.0#1691706645918238...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnative

Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.0.2#1691706676748683...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.1.0#1691706674573693...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.1.1#1691706675331569...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.1.2#1691706679926052...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.2.1#1691706673665963...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4_5.zarr/random1/1.2.0#1691706673949494...
Removing gs://cloud-data-benchmark

Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/0.2.2#1691706696061614...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/1.0.2#1691706697055687...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/1.0.0#1691706692436664...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/0.2.1#1691706693075367...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/1.0.1#1691706692268810...
Removing gs://cloud-data-benchmarks/cloud-data-transfer-benchmarking/cloudnativefiles/random_3.0GB_NetCDF4.nc_0.0MB_lz4hc_5.zarr/random1/1.1.1#1691706694025850...
Removing gs://cloud-da

0